In [ ]:
!htseq-count -f bam -r pos -s no -i gene_id -t gene  RNA-siLuc.bam gencode.gtf > siLuc_count.txt

In [ ]:
!htseq-count -f bam -r pos -s no -i gene_id -t gene  RNA-siLin28a.bam gencode.gtf > siLin28a_count.txt

In [ ]:
!grep transcript gencode.gtf | cut -f9 | sed 's/;//g' | sed 's/ /\t/g' | grep MUSP | cut -f7,8,19,20 | grep MUSP | sed 's/"//g' | sed 's/\./\t/g' | cut -f2,4,7 | uniq > mmu_id_gene.txt

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [93]:
siluc = pd.read_csv('siLuc_count.txt', header = None, sep = '\t', names = ['gene_name', 'siluc_counts'])
silin28a = pd.read_csv('siLin28a_count.txt', header = None, sep = '\t', names = ['gene_name', 'silin_counts'])

In [96]:
mmu = pd.read_csv('mmu_id_gene.txt', header = None, sep = '\t', names = ['gene_name', 'protein_id']).drop_duplicates().reset_index(drop = True)

In [97]:
siLuc = siluc.iloc[:-5,:]
siLuc = siLuc[siLuc['siluc_counts']>=30]
siLin28a = silin28a.iloc[:-5,:]
siLin28a = siLin28a[siLin28a['silin_counts']>=30]

In [98]:
counts = pd.merge(siLuc, siLin28a, how = 'left', on ='gene_name')

In [99]:
counts['log_fc'] = np.log2(counts['silin_counts']) - np.log2(counts['siluc_counts'])
counts = counts.dropna().reset_index(drop = True)

In [100]:
counts = pd.merge(counts, mmu, how = 'inner', on ='gene_name')

In [101]:
upregulated = counts[counts['log_fc'] >= 1].reset_index(drop=True)
downregulated = counts[counts['log_fc'] <=-1].reset_index(drop=True)

In [102]:
upregulated[['gene_name']].drop_duplicates().to_csv('upregulated_gene_names_siLin28a_treated.txt', sep = '\t', index = None, header = None)
upregulated[['protein_id']].drop_duplicates().to_csv('upregulated_pids_siLin28a_treated.txt', sep = '\t', index = None, header = None)

In [103]:
downregulated[['gene_name']].drop_duplicates().to_csv('downregulated_gene_names_siLin28a_treated.txt', sep = '\t', index = None, header = None)
downregulated[['protein_id']].drop_duplicates().to_csv('downregulated_pids_siLin28a_treated.txt', sep = '\t', index = None, header = None)

## Network propagation

In [147]:
np_up = pd.read_csv('binfo_up.txt', sep ='\t', header = None, names = ['protein_id', 'np_score'])
np_up.sort_values(by = 'np_score', ascending = False, inplace = True)
np_up = pd.merge(np_up, counts, how='left', on ='protein_id')
np_up = np_up.reset_index(drop =True)
np_down = pd.read_csv('binfo_down.txt', sep ='\t', header = None, names = ['protein_id', 'np_score'])
np_down.sort_values(by = 'np_score', ascending = False, inplace = True)
np_down = np_down.reset_index(drop =True)
np_down = pd.merge(np_down, counts, how='left', on ='protein_id')

In [148]:
up_seeds = upregulated[['protein_id']].drop_duplicates()
up_seeds['seed'] = 1
down_seeds =downregulated[['protein_id']].drop_duplicates()
down_seeds['seed'] = 1

In [149]:
np_up = pd.merge(np_up, up_seeds, how ='left', on='protein_id').fillna(0)
np_down = pd.merge(np_down, down_seeds, how ='left', on='protein_id').fillna(0)
np_up.to_csv('./binfo_np_up.csv', sep =',', index=None)
np_down.to_csv('./binfo_np_down.csv', sep =',', index=None)

#### Remove seeds

In [150]:
np_up_no_seed = np_up[np_up['seed']==0].reset_index(drop = True)
np_down_no_seed = np_down[np_down['seed']==0].reset_index(drop = True)

#### Crop top 100 genes

In [151]:
np_up_no_seed_t100 = np_up_no_seed.iloc[:100, :]
np_down_no_seed_t100 = np_down_no_seed.iloc[:100, :]
np_up_no_seed_t100.to_csv('./binfo_np_up_no_seed_t100.csv', sep =',', index=None)
np_down_no_seed_t100.to_csv('./binfo_np_down_no_seed_t100.csv', sep =',', index=None)

#### Extract subgraph from template network

In [156]:
template_network = pd.read_csv('protein.links.detailed.v11.0.txt.graph_up0.7_revised.txt', header=None, sep='\t', names=['source', 'target', 'edge'])

In [168]:
temp_genes=pd.concat([np_up[['protein_id']].iloc[:200,:], np_down[['protein_id']].iloc[:200,:]], axis = 0).drop_duplicates().reset_index(drop=True)
temp_genes.columns = ['source']
temp_network = pd.merge(template_network, temp_genes, how='inner', on ='source')
temp_genes.columns = ['target']
temp_network = pd.merge(temp_network, temp_genes, how='inner', on ='target')

In [171]:
temp_network.to_csv('subnetwork.csv', sep =',', index=None)